In [21]:
# !pip install -U sentence-transformers tqdm

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack
import os
import json
from tqdm.auto import tqdm
import pickle

In [ ]:
df = pd.read_csv("../data/appartments.csv")

In [116]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


In [117]:
# df.duplicated().sum()

In [118]:
df.iloc[2].NearbyLocations

"['AIPL Business Club Sector 62', 'Heritage Xperiential Learning School', 'CK Birla Hospital', 'Paras Trinity Mall Sector 63', 'Rapid Metro Station Sector 56']"

In [119]:
df.iloc[2].LocationAdvantages

"{'AIPL Business Club Sector 62': '2.7 Km', 'Heritage Xperiential Learning School': '2 Km', 'CK Birla Hospital': '2.5 Km', 'Paras Trinity Mall Sector 63': '3.5 Km', 'Rapid Metro Station Sector 56': '3.8 Km', 'De Adventure Park': '6.8 Km', 'Golf Course Ext Rd': '99 Meter', 'DoubleTree by Hilton Hotel Gurgaon': '3.6 Km', 'KIIT College of Engineering Sohna Road': '8.4 Km', 'Mehrauli-Gurgaon Road': '11.8 Km', 'Indira Gandhi International Airport': '21.1 Km', 'Nirvana Rd': '160 Meter', 'TERI Golf Course': '8.7 Km'}"

In [120]:
df.iloc[1].PriceDetails

"{'3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,605 - 2,170 sq.ft.', 'price-range': '₹ 2.2 - 3.03 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,248 - 2,670 sq.ft.', 'price-range': '₹ 3.08 - 3.73 Cr'}}"

In [121]:
df.iloc[2].TopFacilities

"['Terrace Garden', 'Gazebo', 'Fountain', 'Amphitheatre', 'Party Lawn', 'Basketball Court', 'Badminton Court', 'Yoga/Meditation Area', 'Indoor Games']"

In [122]:
df[['PropertyName','TopFacilities']]['TopFacilities'][0]

"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"

In [123]:
# def extract_list(s):
#     return re.findall(r"'(.*?)'", s)

# df['TopFacilities'] = df['TopFacilities'].apply(extract_list)

In [124]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


In [125]:
# df['FacilitiesStr'] = df['TopFacilities'].apply(' '.join)

In [126]:
# df['FacilitiesStr'][0]

In [127]:
def parse_facilities(value):
    if isinstance(value, list):
        return [str(x).strip() for x in value]
    if pd.isna(value):
        return []
    s = str(value).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            lst = json.loads(s.replace("'", '"'))
            return [str(x).strip() for x in lst]
        except Exception:
            s2 = s[1:-1]
            return [x.strip().strip("'\"") for x in s2.split(",") if x.strip()]
    if "|" in s:
        return [x.strip() for x in s.split("|")]
    if "," in s:
        return [x.strip() for x in s.split(",")]
    return [s]

In [128]:
df['TopFacilities_list'] = df['TopFacilities'].apply(parse_facilities)

In [129]:
df['TopFacilities_list'].iloc[3]

['Swimming Pool',
 'Volley Ball Court',
 'Aerobics Centre',
 'Card Room',
 'Barbecue',
 'Sauna',
 'Steam Room',
 'Creche/Day care',
 'Skating Rink']

In [130]:
df['TopFacilities_text'] = df['TopFacilities_list'].apply(lambda lst: " ".join([f.replace(" ", "_") for f in lst]))

In [138]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

tfidf_matrix = tfidf_vectorizer.fit_transform(df['TopFacilities_text'])

In [139]:
tfidf_matrix.toarray()[0]

array([0.        , 0.        , 0.        , 0.2149379 , 0.        ,
       0.        , 0.25505047, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [140]:
cosine_sim1 = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [141]:
cosine_sim1.shape

(246, 246)

In [67]:
#  Build SBERT model + embeddings + similarity 
use_sbert = True   
sbert = None
embeddings = None
sim_fac_sbert = None

if use_sbert:
    try:
        from sentence_transformers import SentenceTransformer
        from sklearn.metrics.pairwise import cosine_similarity

        sbert_model_name = 'all-MiniLM-L6-v2' 
        print("Loading SBERT model:", sbert_model_name)
        sbert = SentenceTransformer(sbert_model_name)

        # encode only the facilities text (ensure df['TopFacilities_text'] exists)
        facilities_list = df['TopFacilities_text'].fillna("").tolist()
        embeddings = sbert.encode(facilities_list, convert_to_numpy=True, show_progress_bar=True)

        # precompute similarity matrix (cosine)
        sim_fac_sbert = cosine_similarity(embeddings, embeddings)
        print("SBERT embeddings built. Shape:", embeddings.shape)

    except Exception as e:
        print("SBERT not available or failed to load:", e)
        sbert = None
        embeddings = None
        sim_fac_sbert = None
else:
    print("SBERT usage disabled (use_sbert=False).")
cosine_fac_tfidf = linear_kernel(X_fac_tfidf, X_fac_tfidf)

Loading SBERT model: all-MiniLM-L6-v2


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 8/8 [00:04<00:00,  1.60it/s]

SBERT embeddings built. Shape: (246, 384)


In [137]:
sim_fac_sbert

array([[0.99999976, 0.6778733 , 0.6560598 , ..., 0.761964  , 0.67770773,
        0.75050116],
       [0.6778733 , 1.0000006 , 0.75501084, ..., 0.7670626 , 0.6119141 ,
        0.70301795],
       [0.6560598 , 0.75501084, 1.        , ..., 0.71317613, 0.5621466 ,
        0.63865227],
       ...,
       [0.761964  , 0.7670626 , 0.71317613, ..., 0.9999996 , 0.69735473,
        0.7618519 ],
       [0.67770773, 0.6119141 , 0.5621466 , ..., 0.69735473, 1.        ,
        0.78424627],
       [0.75050116, 0.70301795, 0.63865227, ..., 0.7618519 , 0.78424627,
        0.9999997 ]], shape=(246, 246), dtype=float32)

In [142]:
def recommend_simple(property_name, sim_matrix, top_n=5):
    """
    property_name : name of the property you want recommendations for
    sim_matrix    : cosine_fac_tfidf OR sim_fac_sbert
    top_n         : how many recommendations to return
    """

    # find index
    idx = df.index[df['PropertyName'] == property_name].tolist()[0]

    # similarity scores for that item
    sim_scores = list(enumerate(sim_matrix[idx]))

    # sort high → low
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # skip itself and take top_n
    top_scores = sim_scores[1 : top_n + 1]

    # extract indices & similarity values
    property_indices = [i for i, score in top_scores]
    similarity_values = [score for i, score in top_scores]

    # build dataframe
    return pd.DataFrame({
        "PropertyName": df.loc[property_indices, "PropertyName"].values,
        "SimilarityScore": similarity_values
    })


In [143]:
recommend_simple("M3M Crown", cosine_sim1)    # TF-IDF based

,PropertyName,SimilarityScore
0,DLF The Ultima,0.276687
1,Ireo Victory Valley,0.246446
2,Emaar MGF Marbella,0.194079
3,BPTP Amstoria,0.192640
4,Sobha City,0.179286


In [144]:
recommend_simple("M3M Crown", sim_fac_sbert)       # SBERT based

,PropertyName,SimilarityScore
0,Ireo Victory Valley,0.828760
1,DLF The Ultima,0.827552
2,Central Park Flower Valley Mikasa Plots,0.827198
3,M3M Sky Lofts,0.823463
4,Emaar Mgf Palm Terraces Select,0.820544


In [73]:
def recommend_combined(property_name, tfidf_sims, sbert_sims, top_n=5, alpha=0.5):
    """
    alpha = weight for TF-IDF (0.5 = equal weight)
    """

    idx = df.index[df['PropertyName'] == property_name].tolist()[0]

    # raw score arrays
    tfidf_score = np.array(tfidf_sims[idx])
    sbert_score = np.array(sbert_sims[idx])

    # min-max normalize
    t_norm = (tfidf_score - tfidf_score.min()) / (tfidf_score.max() - tfidf_score.min() + 1e-9)
    s_norm = (sbert_score - sbert_score.min()) / (sbert_score.max() - sbert_score.min() + 1e-9)

    # fuse scores
    final_score = alpha * t_norm + (1 - alpha) * s_norm

    # Sort & pick top_n
    order = final_score.argsort()[::-1]
    order = [i for i in order if i != idx][:top_n]

    return pd.DataFrame({
        "PropertyName": df.loc[order, "PropertyName"].values,
        "CombinedScore": final_score[order]
    })


In [74]:
recommend_combined("M3M Crown", cosine_fac_tfidf, sim_fac_sbert, top_n=5, alpha=0.6)

,PropertyName,CombinedScore
0,DLF The Ultima,0.481148
1,Ireo Victory Valley,0.463598
2,Emaar MGF Marbella,0.408826
3,Sobha City,0.406210
4,BPTP Pedestal,0.402257


### we will use sim_fac_sbert model 

In [81]:
df_appartments = pd.read_csv("data/appartments.csv")

In [82]:
df_appartments.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


In [83]:
df_appartments['PriceDetails'][10]

"{'2 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,055 sq.ft.', 'price-range': '₹ 1.05 - 1.5 Cr'}, '3 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,325 - 1,525 sq.ft.', 'price-range': '₹ 1.35 - 1.84 Cr'}}"

In [84]:
def refined_parse_modified_v2(detail_str):
    try:
        details = json.loads(detail_str.replace("'", "\""))
    except:
        return {}

    extracted = {}
    for bhk, detail in details.items():
        # Extract building type
        extracted[f'building type_{bhk}'] = detail.get('building_type')

        # Parsing area details
        area = detail.get('area', '')
        area_parts = area.split('-')
        if len(area_parts) == 1:
            try:
                value = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area low {bhk}'] = value
                extracted[f'area high {bhk}'] = value
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None
        elif len(area_parts) == 2:
            try:
                extracted[f'area low {bhk}'] = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area high {bhk}'] = float(area_parts[1].replace(',', '').replace(' sq.ft.', '').strip())
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None

        # Parsing price details
        price_range = detail.get('price-range', '')
        price_parts = price_range.split('-')
        if len(price_parts) == 2:
            try:
                extracted[f'price low {bhk}'] = float(price_parts[0].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                extracted[f'price high {bhk}'] = float(price_parts[1].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                if 'L' in price_parts[0]:
                    extracted[f'price low {bhk}'] /= 100
                if 'L' in price_parts[1]:
                    extracted[f'price high {bhk}'] /= 100
            except:
                extracted[f'price low {bhk}'] = None
                extracted[f'price high {bhk}'] = None

    return extracted
# Apply the refined parsing and generate the new DataFrame structure
data_refined = []

for _, row in df_appartments.iterrows():
    features = refined_parse_modified_v2(row['PriceDetails'])
    
    # Construct a new row for the transformed dataframe
    new_row = {'PropertyName': row['PropertyName']}
    
    # Populate the new row with extracted features
    for config in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
        new_row[f'building type_{config}'] = features.get(f'building type_{config}')
        new_row[f'area low {config}'] = features.get(f'area low {config}')
        new_row[f'area high {config}'] = features.get(f'area high {config}')
        new_row[f'price low {config}'] = features.get(f'price low {config}')
        new_row[f'price high {config}'] = features.get(f'price high {config}')
    
    data_refined.append(new_row)

df_final_refined_v2 = pd.DataFrame(data_refined).set_index('PropertyName')

In [85]:
df_final_refined_v2['building type_Land'] = df_final_refined_v2['building type_Land'].replace({'':'Land'})

In [86]:
df['PriceDetails'][10]

"{'2 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,055 sq.ft.', 'price-range': '₹ 1.05 - 1.5 Cr'}, '3 BHK': {'building_type': 'Independent Floor', 'area_type': 'Carpet Area', 'area': '1,325 - 1,525 sq.ft.', 'price-range': '₹ 1.35 - 1.84 Cr'}}"

In [87]:
df_final_refined_v2

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,...,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,None,NaN,NaN,NaN,NaN,Apartment,1370.0,1370.0,2.0000,2.40,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
M3M Crown,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Adani Brahma Samsara Vilasa,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,Land,500.0,4329.0,2.05,41.13
Sobha City,None,NaN,NaN,NaN,NaN,Apartment,1381.0,1692.0,1.5500,3.21,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Signature Global City 93,None,NaN,NaN,NaN,NaN,Independent Floor,981.0,1118.0,0.9301,1.06,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,None,NaN,NaN,NaN,NaN,Apartment,964.0,964.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Pyramid Urban Homes 2,Apartment,335.0,398.0,23.45,0.2786,Apartment,500.0,625.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Satya The Hermitage,None,NaN,NaN,NaN,NaN,Apartment,1450.0,1450.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [88]:
categorical_columns = df_final_refined_v2.select_dtypes(include=['object']).columns.tolist()

In [89]:
categorical_columns

['building type_1 BHK',
 'building type_2 BHK',
 'building type_3 BHK',
 'building type_4 BHK',
 'building type_5 BHK',
 'building type_6 BHK',
 'building type_1 RK',
 'building type_Land']

In [90]:
ohe_df = pd.get_dummies(df_final_refined_v2, columns=categorical_columns, drop_first=True)

In [91]:
ohe_df.fillna(0,inplace=True)

In [92]:
ohe_df

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.0,0.0,0.00,0.0000,1370.0,1370.0,2.0000,2.40,1850.0,2050.0,...,False,False,False,False,False,False,False,False,False,False
M3M Crown,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.00,1605.0,2170.0,...,False,False,False,False,False,False,False,False,False,False
Adani Brahma Samsara Vilasa,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.00,1800.0,3150.0,...,False,False,True,False,False,True,False,False,False,False
Sobha City,0.0,0.0,0.00,0.0000,1381.0,1692.0,1.5500,3.21,1711.0,2343.0,...,False,False,False,False,False,False,False,False,False,False
Signature Global City 93,0.0,0.0,0.00,0.0000,981.0,1118.0,0.9301,1.06,1235.0,1530.0,...,True,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.0,0.0,0.00,0.0000,964.0,964.0,0.0000,0.00,1127.0,1127.0,...,False,False,False,False,False,False,False,False,False,False
Pyramid Urban Homes 2,335.0,398.0,23.45,0.2786,500.0,625.0,0.0000,0.00,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
Satya The Hermitage,0.0,0.0,0.00,0.0000,1450.0,1450.0,0.0000,0.00,1991.0,1991.0,...,False,False,False,False,False,False,False,False,False,False


In [93]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
ohe_df_normalized = pd.DataFrame(scaler.fit_transform(ohe_df), columns=ohe_df.columns, index=ohe_df.index)

In [94]:
ohe_df_normalized.head()

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-0.252266,-0.169584,-0.105197,-0.082332,1.223499,1.020101,-0.173712,1.158423,0.553787,0.370864,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Crown,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.293086,0.472749,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Adani Brahma Samsara Vilasa,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.500583,1.304803,...,-0.289310,-0.063888,2.683282,-0.063888,-0.171139,3.924283,-0.236208,-0.111111,-0.216353,-0.063888
Sobha City,-0.252266,-0.169584,-0.105197,-0.082332,1.240497,1.470610,-0.198425,1.680336,0.405879,0.619632,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Signature Global City 93,-0.252266,-0.169584,-0.105197,-0.082332,0.622383,0.667529,-0.232468,0.295011,-0.100626,-0.070634,...,3.456497,-0.063888,2.683282,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [98]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim2 = cosine_similarity(ohe_df_normalized)

In [99]:
cosine_sim2.shape

(246, 246)

In [100]:
def recommend_properties_with_scores(property_name, top_n=246):
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim2[ohe_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = ohe_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Golf Hills')

,PropertyName,SimilarityScore
0,AIPL The Peaceful Homes,0.955462
1,Smartworld One DXP,0.954670
2,Unitech Escape,0.953092
3,M3M Capital,0.951156
4,BPTP Terra,0.943128
...,...,...
240,Golden Park,-0.522391
241,Satya Merano Greens,-0.523660
242,ROF Normanton Park,-0.525129
243,BPTP Green Oaks,-0.525286


In [101]:
df = pd.read_csv('data/appartments.csv')

In [102]:
df[['PropertyName','LocationAdvantages']]['LocationAdvantages'][0]

"{'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}"

In [103]:
import ast
import re
import pandas as pd
import numpy as np

# --- helper functions ---
def safe_parse_dict(s):
    """Safely parse a string representation of a dictionary to a Python dict.
    Returns {} on any failure (so downstream code doesn't crash)."""
    if pd.isna(s):
        return {}
    try:
        # some rows might already be dicts
        if isinstance(s, dict):
            return s
        return ast.literal_eval(s)
    except Exception:
        # fallback: try crude parsing (key: value pairs)
        try:
            s = str(s).strip()
            s = s.strip('{} ')
            items = [it for it in s.split(", '") if ':' in it or "'" in it]
            out = {}
            for item in items:
                # try splitting on the first colon
                if ":" in item:
                    k, v = item.split(":", 1)
                    k = k.strip().strip(" '\"")
                    v = v.strip().strip(" '\"")
                    out[k] = v
            return out
        except Exception:
            return {}

_num_re = re.compile(r"[-+]?\d[\d,\.]*")  # find number like 1,234.5 or 2.5

def distance_to_meters_raw(distance_str):
    """Convert a distance string to meters (float).
    Examples: '2.5 KM' -> 2500.0, '800 Meter' -> 800.0
    Returns np.nan if can't parse.
    """
    if distance_str is None:
        return np.nan
    text = str(distance_str).strip()
    if text == "":
        return np.nan
    text_low = text.lower()

    # find numeric piece
    m = _num_re.search(text_low)
    if not m:
        return np.nan
    num_str = m.group(0).replace(",", "")
    try:
        num = float(num_str)
    except Exception:
        return np.nan

    # unit detection (prefer explicit units)
    # treat any mention of 'km', 'kilomet' as kilometres
    if 'km' in text_low or 'kilomet' in text_low:
        return num * 1000.0
    # meter synonyms
    if 'm' in text_low or 'meter' in text_low or 'metre' in text_low:
        # careful: avoid matching 'km' here because we already returned for 'km'
        return num if num > 0 else np.nan

    # no explicit unit: assume meters (safer default)
    return num

def normalize_location_name(name):
    """Basic normalization for location names to reduce duplicate variations.
    - lowercases
    - removes extra whitespace
    - drops trailing/leading commas and periods
    - expands common abbreviations if you add mappings
    """
    if not isinstance(name, str):
        return name
    s = name.strip()
    s = re.sub(r"[\.\,]+$", "", s)       # remove trailing punctuation
    s = re.sub(r"\s+", " ", s)           # collapse multiple spaces
    s = s.strip()
    return s

# --- main pipeline function ---
def expand_locationadvantages_to_wide(df,
                                     source_col='LocationAdvantages',
                                     index_col='PropertyName',
                                     fillna_value=None):
    """
    Parse + clean the LocationAdvantages column and return a wide DataFrame:
    one column per location (values in meters).
    """
    # 1) parse each string into dict
    parsed = df[source_col].apply(safe_parse_dict)

    # 2) convert each dict's values to meters and normalize keys
    def convert_and_normalize(d):
        out = {}
        if not d:
            return out
        for k, v in d.items():
            if k is None:
                continue
            k_clean = normalize_location_name(k)
            m = distance_to_meters_raw(v)
            out[k_clean] = m
        return out

    meters_dicts = parsed.apply(convert_and_normalize)

    # 3) create wide DataFrame
    location_wide = pd.DataFrame.from_records(meters_dicts.tolist(), index=df.index)

    # 4) optionally set property names as index (if exists)
    if index_col in df.columns:
        location_wide.index = df[index_col]

    # 5) fillna if requested
    if fillna_value is not None:
        location_wide = location_wide.fillna(fillna_value)

    return location_wide

# --- run pipeline on your df (example) ---
# set fillna_value to 54000 as you did before (means "very far / missing")
location_df = expand_locationadvantages_to_wide(df, fillna_value=54000)

# quick checks
print("shape:", location_df.shape)
display(location_df.head())


shape: (246, 1066)


,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
M3M Crown,54000.0,54000.0,54000.0,54000.0,54000.0,4400.0,54000.0,54000.0,54000.0,8200.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Adani Brahma Samsara Vilasa,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Sobha City,54000.0,54000.0,54000.0,54000.0,54000.0,9200.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Signature Global City 93,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0


In [104]:
#Merge Similar Location Columns (Fuzzy Matching) ---
import pandas as pd
import difflib
from collections import defaultdict

def normalize_col(c):
    c = str(c).lower().strip()
    c = re.sub(r'[^a-z0-9 ]+', ' ', c)
    c = re.sub(r'\s+', ' ', c)
    return c

def merge_similar_columns(df, threshold=85, agg='min'):
    cols = list(df.columns)
    norm_cols = {c: normalize_col(c) for c in cols}

    # group similar names
    groups = []
    used = set()

    for c in cols:
        if c in used:
            continue

        group = [c]
        used.add(c)

        for other in cols:
            if other in used:
                continue

            ratio = difflib.SequenceMatcher(None, norm_cols[c], norm_cols[other]).ratio() * 100

            if ratio >= threshold:
                group.append(other)
                used.add(other)

        groups.append(group)

    # merge groups
    merged_df = df.copy()
    mapping = {}

    for group in groups:
        if len(group) == 1:
            mapping[group[0]] = group
            continue

        # canonical = longest name (more descriptive)
        canonical = max(group, key=len)
        mapping[canonical] = group

        if agg == 'min':
            merged = merged_df[group].min(axis=1)
        elif agg == 'mean':
            merged = merged_df[group].mean(axis=1)
        else:
            merged = merged_df[group].min(axis=1)

        merged_df[canonical] = merged

        # drop other duplicate columns
        for col in group:
            if col != canonical:
                merged_df.drop(columns=[col], inplace=True)

    return merged_df, mapping

# ---- RUN MERGE ----
location_df_cleaned, merge_map = merge_similar_columns(location_df, threshold=85, agg='min')

print("Original column count:", len(location_df.columns))
print("Cleaned column count :", len(location_df_cleaned.columns))

print("\nMerged Groups:")
for k, v in merge_map.items():
    if len(v) > 1:
        print(f"{k} <- {v}")


Original column count: 1066
Cleaned column count : 853

Merged Groups:
Paras Hospitals <- ['Park Hospital', 'Arc Hospital', 'Paras Hospital', 'Paras Hospitals']
Gurgaon Old Railway Station <- ['Gurgaon Railway Station', 'Gurgaon Old Railway Station', 'Gurugram Railway Station', 'Gurgaon railway station']
Dwarka Sector 21, Metro Station <- ['Dwarka Sector 21, Metro Station', 'Dwarka Sector 21 Metro Station', 'Dwarka Sector 21 Metro station', 'Dwarka sector 21 metro station']
Indira Gandhi International Airport <- ['Indira Gandhi International Airport', 'Indira Gandhi Intl Airport']
Fun N Food Water Park <- ['Fun N Food Waterpark', 'Fun N Food Water Park', 'Fun N Food WaterPark']
Manipal Hospital, Palam Vihar <- ['Park Hospital, Palam Vihar', 'Metro Hospital, Palam Vihar', 'Manipal Hospital, Palam Vihar']
Dwaraka Expressway <- ['Dwarka Expressway', 'Dwaraka Expressway', 'Dwarka expressway']
Tau Devi Lal Sports Complex <- ['Tau DeviLal Sports Complex', 'Tau Devi Lal Sports Complex']
Vipul

In [106]:
location_df_cleaned.fillna(54000,inplace=True)

In [107]:
location_df_cleaned

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,...,Shri Hanuman Ji Mandir,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,800.0,2500.0,3100.0,5400.0,1200.0,7700.0,7200.0,7400.0,14700.0,6200.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
M3M Crown,54000.0,54000.0,54000.0,4400.0,54000.0,54000.0,8100.0,8200.0,14100.0,5000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Adani Brahma Samsara Vilasa,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,21100.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Sobha City,54000.0,54000.0,54000.0,9200.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Signature Global City 93,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,15200.0,54000.0,...,54000.0,54000.0,3900.0,4000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0
Pyramid Urban Homes 2,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,29400.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,3100.0,3700.0,6100.0,7700.0,3000.0,54000.0
Satya The Hermitage,54000.0,54000.0,54000.0,8800.0,54000.0,54000.0,54000.0,54000.0,21700.0,54000.0,...,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0,54000.0


In [108]:
from sklearn.preprocessing import StandardScaler
# Initialize the scaler
scaler = StandardScaler()

# Apply the scaler to the entire dataframe
location_df_normalized = pd.DataFrame(scaler.fit_transform(location_df_cleaned), columns=location_df_cleaned.columns, index=location_df_cleaned.index)

In [109]:
location_df_normalized

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,Indira Gandhi International Airport,Hamoni Golf Camp,...,Shri Hanuman Ji Mandir,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-7.960979,-15.652476,-15.652476,-3.147217,-3.605767,-10.231739,-5.396042,-6.023233,-1.410037,-5.664722,...,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888
M3M Crown,0.128476,0.063888,0.063888,-3.218882,0.289192,0.090308,-5.288526,-5.916893,-1.450605,-5.811524,...,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888
Adani Brahma Samsara Vilasa,0.128476,0.063888,0.063888,0.335688,0.289192,0.090308,0.194781,0.171073,-0.977313,0.182906,...,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888
Sobha City,0.128476,0.063888,0.063888,-2.874891,0.289192,0.090308,0.194781,0.171073,1.247161,0.182906,...,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888
Signature Global City 93,0.128476,0.063888,0.063888,0.335688,0.289192,0.090308,0.194781,0.171073,1.247161,0.182906,...,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.128476,0.063888,0.063888,0.335688,0.289192,0.090308,0.194781,0.171073,-1.376230,0.182906,...,0.063888,0.063888,-15.652476,-15.652476,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888
Pyramid Urban Homes 2,0.128476,0.063888,0.063888,0.335688,0.289192,0.090308,0.194781,0.171073,-0.416123,0.182906,...,0.063888,0.063888,0.063888,0.063888,-15.652476,-15.652476,-15.652476,-15.652476,-15.652476,0.063888
Satya The Hermitage,0.128476,0.063888,0.063888,-2.903557,0.289192,0.090308,0.194781,0.171073,-0.936745,0.182906,...,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888


In [110]:
cosine_sim3 = cosine_similarity(location_df_normalized)

In [111]:
cosine_sim3.shape

(246, 246)

In [146]:
cosine_sim2.shape

(246, 246)

In [147]:
cosine_sim1.shape

(246, 246)

In [149]:
def recommend_properties_with_scores(property_name, top_n=246):
    
    cosine_sim_matrix = 30*cosine_sim1 + 20*cosine_sim2 + 8*cosine_sim3
    # cosine_sim_matrix = cosine_sim3
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Crown')

,PropertyName,SimilarityScore
0,DLF The Ultima,26.981097
1,Bestech Altura,23.221058
2,Emaar Gurgaon Greens,21.172758
3,DLF The Primus,20.185795
4,Bestech Park View Sanskruti,20.078044
...,...,...
240,Vatika Express City,-7.421612
241,BPTP Freedom Park Life,-7.491964
242,Central Park Flower Valley The Room,-7.551950
243,Ace Palm Floors,-7.774732


In [ ]:
cosine_sim1
cosine_sim2
cosine_sim3
location_df_cleaned

In [153]:
import pickle

with open("website/dataset/Top_facilities.pkl", "wb") as f:
    pickle.dump(sim_fac_sbert, f)

with open("website/dataset/price_based.pkl", "wb") as f:
    pickle.dump(cosine_sim2, f)

with open("website/dataset/location_based.pkl", "wb") as f:
    pickle.dump(cosine_sim3, f)

with open("website/dataset/distance_location.pkl", "wb") as f:
    pickle.dump(location_df_cleaned, f)

In [ ]:
cosine_sim3.shape

In [150]:
def recommend_properties_with_scores(property_name, top_n=246):
    
    # cosine_sim_matrix = 30*cosine_sim1 + 20*cosine_sim2 + 8*cosine_sim3
    cosine_sim_matrix = cosine_sim3
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Crown')

,PropertyName,SimilarityScore
0,International City by SOBHA Phase 2,0.216136
1,Smartworld One DXP,0.150921
2,Ambience Creacions,0.134112
3,Rishali Luxe Residency 112,0.123494
4,Zara Rossa,0.109435
...,...,...
240,Suncity Avenue 76,-0.025105
241,BPTP Green Oaks,-0.025760
242,Godrej Air,-0.026772
243,Ansal API Esencia,-0.042096


In [150]:
def recommend_properties_with_scores(property_name, top_n=246):
    
    # cosine_sim_matrix = 30*cosine_sim1 + 20*cosine_sim2 + 8*cosine_sim3
    cosine_sim_matrix = cosine_sim3
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Crown')

,PropertyName,SimilarityScore
0,International City by SOBHA Phase 2,0.216136
1,Smartworld One DXP,0.150921
2,Ambience Creacions,0.134112
3,Rishali Luxe Residency 112,0.123494
4,Zara Rossa,0.109435
...,...,...
240,Suncity Avenue 76,-0.025105
241,BPTP Green Oaks,-0.025760
242,Godrej Air,-0.026772
243,Ansal API Esencia,-0.042096


In [152]:
def recommend_properties_with_scores(property_name, top_n=246):
    
    # cosine_sim_matrix = 30*cosine_sim1 + 20*cosine_sim2 + 8*cosine_sim3
    cosine_sim_matrix = cosine_sim1
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Crown')

,PropertyName,SimilarityScore
0,DLF The Ultima,0.276687
1,Ireo Victory Valley,0.246446
2,Emaar MGF Marbella,0.194079
3,BPTP Amstoria,0.192640
4,Sobha City,0.179286
...,...,...
240,Yashika 104,0.000000
241,Zara Rossa,0.000000
242,Pegasus Atulyam 83,0.000000
243,Suncity Avenue 102,0.000000


In [154]:
def recommend_properties_with_scores(property_name, top_n=246):
    
    # cosine_sim_matrix = 30*cosine_sim1 + 20*cosine_sim2 + 8*cosine_sim3
    cosine_sim_matrix = sim_fac_sbert
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Crown')

,PropertyName,SimilarityScore
0,Ireo Victory Valley,0.828760
1,DLF The Ultima,0.827552
2,Central Park Flower Valley Mikasa Plots,0.827198
3,M3M Sky Lofts,0.823463
4,Emaar Mgf Palm Terraces Select,0.820544
...,...,...
240,Pyramid Urban Homes 2,0.509355
241,JMS The Nation,0.498488
242,Optimal ultra luxury builder floors,0.485030
243,Pegasus Atulyam 83,0.442366


In [155]:
df = pd.read_csv("data/appartments.csv")

In [156]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


In [157]:
df = df[["PropertyName","Link"]]

In [158]:
df.head()

,PropertyName,Link
0,Smartworld One DXP,https://www.99acres.com/smartworld-one-dxp-sec...
1,M3M Crown,https://www.99acres.com/m3m-crown-sector-111-g...
2,Adani Brahma Samsara Vilasa,https://www.99acres.com/adani-brahma-samsara-v...
3,Sobha City,https://www.99acres.com/sobha-city-sector-108-...
4,Signature Global City 93,https://www.99acres.com/signature-global-city-...


In [159]:
import pickle

with open("website/dataset/url.pkl", "wb") as f:
    pickle.dump(df, f)